In [2]:
import os
import psycopg2
from pathlib import Path

# Database connection parameters
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")

def get_connection():
    """Create and return a database connection"""
    return psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host
    )

def save_to_file(content, filename):
    """Save content to a file in the current working directory"""
    # Use current working directory instead of __file__
    file_path = Path(os.getcwd()) / filename
    with open(file_path, 'w') as f:
        f.write(content)
    print(f"Created: {filename}")

def extract_tables():
    """Extract table definitions including constraints"""
    with get_connection() as conn:
        with conn.cursor() as cur:
            # Get all tables in the public schema
            cur.execute("""
                SELECT table_name 
                FROM information_schema.tables 
                WHERE table_schema = 'public' 
                AND table_type = 'BASE TABLE'
            """)
            tables = cur.fetchall()
            
            for table in tables:
                table_name = table[0]
                # Get table creation SQL including constraints
                cur.execute(f"""
                    SELECT 
                        'CREATE TABLE ' || tablename || E'\n(\n' ||
                        string_agg(
                            '    ' || column_name || ' ' || data_type || 
                            COALESCE(' ' || domain_name, '') ||
                            CASE WHEN character_maximum_length IS NOT NULL 
                                THEN '(' || character_maximum_length || ')' 
                                ELSE '' 
                            END ||
                            CASE WHEN is_nullable = 'NO' 
                                THEN ' NOT NULL' 
                                ELSE '' 
                            END,
                            E',\n'
                        ) || 
                        E'\n);\n' as create_table
                    FROM (
                        SELECT 
                            c.table_name AS tablename,
                            c.column_name,
                            c.data_type,
                            c.character_maximum_length,
                            c.is_nullable,
                            c.domain_name
                        FROM information_schema.columns c
                        WHERE c.table_name = %s
                        AND c.table_schema = 'public'
                        ORDER BY c.ordinal_position
                    ) AS t
                    GROUP BY tablename;
                """, (table_name,))
                
                create_table_sql = cur.fetchone()[0]
                
                # Get primary key constraints
                cur.execute("""
                    SELECT pg_get_constraintdef(con.oid)
                    FROM pg_constraint con
                    INNER JOIN pg_class rel ON rel.oid = con.conrelid
                    WHERE rel.relname = %s
                    AND con.contype = 'p'
                """, (table_name,))
                pk_constraints = cur.fetchall()
                
                # Get foreign key constraints
                cur.execute("""
                    SELECT pg_get_constraintdef(con.oid)
                    FROM pg_constraint con
                    INNER JOIN pg_class rel ON rel.oid = con.conrelid
                    WHERE rel.relname = %s
                    AND con.contype = 'f'
                """, (table_name,))
                fk_constraints = cur.fetchall()
                
                # Add constraints to create table SQL
                for pk in pk_constraints:
                    create_table_sql = create_table_sql.rstrip(';\n') + ',\n    ' + pk[0] + ';\n'
                
                # Add ALTER TABLE statements for foreign keys
                for fk in fk_constraints:
                    create_table_sql += f"ALTER TABLE {table_name} ADD CONSTRAINT {fk[0]};\n"
                
                save_to_file(create_table_sql, f'table_{table_name}.sql')

def extract_views():
    """Extract view definitions"""
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT viewname, definition 
                FROM pg_views 
                WHERE schemaname = 'public'
            """)
            views = cur.fetchall()
            
            for view_name, view_def in views:
                save_to_file(view_def + ';\n', f'view_{view_name}.sql')

def extract_functions():
    """Extract function definitions"""
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT proname, pg_get_functiondef(oid) 
                FROM pg_proc 
                WHERE pronamespace = 'public'::regnamespace
            """)
            functions = cur.fetchall()
            
            for func_name, func_def in functions:
                save_to_file(func_def, f'function_{func_name}.sql')

def main():
    try:
        print(f"Starting extraction to: {os.getcwd()}")
        extract_tables()
        extract_views()
        extract_functions()
        print("Successfully extracted database objects to SQL files")
    except Exception as e:
        print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Starting extraction to: c:\Users\seanj\Documents\Projects\Vestia\vestia_database
Created: table_account.sql
Created: table_client.sql
Created: table_asset_price.sql
Created: table_corporate_action.sql
Created: table_cash_trade.sql
Created: table_managed_portfolio.sql
Created: table_asset_trade.sql
Created: table_asset.sql
Created: table_account_performance.sql
Created: table_instruction.sql
Created: table_client_performance.sql
Created: table_admin.sql
Created: view_vw_cash_balance.sql
Created: view_vw_latest_price.sql
Created: view_vw_asset_balance.sql
Created: view_vw_client_performance.sql
Created: function_update_date_updated_column.sql
Created: function_get_account_performance.sql
Created: function_get_client_performance.sql
Created: function_get_account_holdings.sql
Created: function_get_assets.sql
Created: function_get_client_information.sql
Created: function_post_new_account.sql
Created: function_get_account_summary.sql
Created: function_get_client_payments.sql
Created: functio